# Experiment 01

This Experiment consists in get the Four benchmarks results proposed by Niall O’Connor (https://link.springer.com/chapter/10.1007/1-84628-224-1_6),
The four benchmarks are:

- 1-Day Lag : The predicted value is the same as the previous day.
- SMA5      : The predicet value is a Simple Moving Average from the 5 Previous Days.
- SMA10     : The predicet value is a Simple Moving Average from the 10 Previous Days.
- SMA20     : The predicet value is a Simple Moving Average from the 20 Previous Days.

In [1]:
import numpy as np
import pandas as pd

import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import warnings
warnings.filterwarnings('ignore')

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

### Constants and Parameters

In [2]:
dfColumns = ['date','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']
col15 = ['open','d-5_high','d-5_low','d-4_high','d-4_low','d-3_high','d-3_low','d-2_high','d-2_low','d-1_high','d-1_low','EMA_high','EMA_low','UB_close','LB_close']
col25 = ['open','d-5_open','d-5_high','d-5_low','d-5_close','d-4_open','d-4_high','d-4_low','d-4_close','d-3_open','d-3_high','d-3_low','d-3_close','d-2_open','d-2_high','d-2_low','d-2_close','d-1_open','d-1_high','d-1_low','d-1_close','EMA_high' ,'EMA_low','UB_close','LB_close']
DAYS = 5
TEST_BEGIN = '2021.04.06'
TEST_END = '2021.06.01'
TRAIN_BEGIN = '2020.03.31'
TRAIN_END = '2021.04.05'
TIMES = 10

#### Opening the dataset

In [3]:
df = pd.read_csv('../Data/PETR4_Daily.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

df2 = pd.read_csv('../Data/VALE3_Daily.csv', sep = "\t")

df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

df 
df.tail(290)

,date,open,high,low,close
1049,2020.03.31,13.16,14.07,13.16,13.54
1050,2020.04.01,13.01,14.08,12.89,13.84
1051,2020.04.02,14.90,16.01,14.52,15.01
1052,2020.04.03,15.77,15.83,14.44,14.84
1053,2020.04.06,15.53,15.58,14.69,15.26
...,...,...,...,...,...
1334,2021.05.26,25.91,26.18,25.65,26.09
1335,2021.05.27,26.08,26.25,25.77,25.91
1336,2021.05.28,26.20,27.02,26.15,26.99
1337,2021.05.31,26.96,27.20,26.78,26.87


In [4]:
#PETR4
df = calculateEMA(df,DAYS,newColumns)
df = calculateBB(df,DAYS,newColumns)
df = addPreviousDays(df,DAYS, newColumns)

#VALE3
df2 = calculateEMA(df2,DAYS,newColumns)
df2 = calculateBB(df2,DAYS,newColumns)
df2 = addPreviousDays(df2,DAYS, newColumns)

In [5]:
##df = getPeriod(df, '2020.03.02','2021.04.30', True)
##initialDf = getPeriod(df, '2020.03.02','2021.04.30', True)

In [6]:
initialDf_PETR4 = getPeriod(df, '2019.01.02','2021.06.01', True)
initialDf_VALE3 = getPeriod(df2, '2019.01.02','2021.06.01', True)

In [7]:
initialDf_PETR4

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2019.01.02,21.18,22.73,20.93,22.60,20.609175,21.499077,20.380957,21.271656,21.776315,...,19.18,20.37,19.55,20.08,19.41,19.69,20.38,20.39,19.50,19.63
1,2019.01.03,22.51,23.32,22.36,23.16,21.242783,22.106051,21.040638,21.901104,23.157151,...,19.99,20.36,19.34,20.37,19.18,20.37,19.55,20.08,19.41,19.69
2,2019.01.04,23.35,23.43,22.99,23.22,21.945189,22.547367,21.690425,22.340736,24.186721,...,20.74,21.31,20.22,20.65,19.99,20.36,19.34,20.37,19.18,20.37
3,2019.01.07,23.35,24.35,23.20,23.59,22.413459,23.148245,22.193617,22.757157,24.644733,...,20.93,22.60,20.77,21.45,20.74,21.31,20.22,20.65,19.99,20.36
4,2019.01.08,23.86,23.88,23.27,23.45,22.895640,23.392163,22.552411,22.988105,24.953473,...,22.36,23.16,21.18,22.73,20.93,22.60,20.77,21.45,20.74,21.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,2021.05.26,25.91,26.18,25.65,26.09,26.114791,26.354219,25.760657,26.042263,26.608719,...,25.94,25.95,26.15,26.28,25.72,25.93,25.85,26.35,25.80,26.15
595,2021.05.27,26.08,26.25,25.77,25.91,26.103194,26.319479,25.763771,25.998175,26.604087,...,25.99,26.39,26.08,26.31,25.94,25.95,26.15,26.28,25.72,25.93
596,2021.05.28,26.20,27.02,26.15,26.99,26.135462,26.552986,25.892514,26.328784,26.617551,...,25.78,25.84,26.28,26.52,25.99,26.39,26.08,26.31,25.94,25.95
597,2021.05.31,26.96,27.20,26.78,26.87,26.410308,26.768657,26.188343,26.509189,27.147657,...,25.65,26.09,26.48,26.62,25.78,25.84,26.28,26.52,25.99,26.39


In [8]:
initialDf_VALE3

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2019.01.02,45.72,46.96,45.52,46.71,45.965361,46.778539,45.580846,46.324671,46.091838,...,45.16,46.11,45.97,46.85,45.91,46.50,46.39,46.78,45.21,45.54
1,2019.01.03,46.44,46.57,44.25,44.80,46.123574,46.709026,45.137231,45.816447,46.532241,...,45.05,45.25,45.81,46.35,45.16,46.11,45.97,46.85,45.91,46.50
2,2019.01.04,45.55,47.95,45.55,47.71,45.932382,47.122684,45.274820,46.447631,46.581616,...,45.94,46.63,45.80,46.07,45.05,45.25,45.81,46.35,45.16,46.11
3,2019.01.07,48.34,49.05,47.28,47.46,46.734922,47.765123,45.943214,46.785088,48.670273,...,45.52,46.71,45.99,46.81,45.94,46.63,45.80,46.07,45.05,45.25
4,2019.01.08,47.72,48.27,47.19,47.91,47.063281,47.933415,46.358809,47.160058,49.216909,...,44.25,44.80,45.72,46.96,45.52,46.71,45.99,46.81,45.94,46.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,2021.05.26,106.37,110.32,106.03,110.20,109.166674,111.131617,107.622606,109.807900,113.973861,...,108.50,109.40,111.70,112.10,110.20,111.11,112.10,112.78,109.84,112.25
595,2021.05.27,110.80,112.09,109.77,110.99,109.711116,111.451078,108.338404,110.201933,113.405981,...,107.51,109.78,111.11,111.25,108.50,109.40,111.70,112.10,110.20,111.11
596,2021.05.28,111.11,112.75,109.42,111.59,110.177411,111.884052,108.698936,110.664622,113.405981,...,106.78,107.05,108.00,110.14,107.51,109.78,111.11,111.25,108.50,109.40
597,2021.05.31,112.50,114.89,112.50,114.78,110.951607,112.886035,109.965957,112.036415,114.811204,...,106.03,110.20,110.28,110.89,106.78,107.05,108.00,110.14,107.51,109.78


#### Experiment 01: Benchmark

In [9]:
def exp01(df):
    df = df[['date','open', 'high', 'low', 'close']]
    
    df = df[['date','open', 'high', 'low', 'close']]

    minaux = df['low'].copy()
    maxaux = df['high'].copy()

    df['maxonedaylag'] = df['high'].shift(1)
    df['minonedaylag'] = df['low'].shift(1)

    df['maxSma5'] = df['high'].rolling(window=5).mean()
    df['minSma5'] = df['low'].rolling(window=5).mean()

    df['maxSma10'] = df['high'].rolling(window=10).mean()
    df['minSma10'] = df['low'].rolling(window=10).mean()

    df['maxSma20'] = df['high'].rolling(window=20).mean()
    df['minSma20'] = df['low'].rolling(window=20).mean()
    
    df = getPeriod(df, TEST_BEGIN, TEST_END)
    
    return df

def printExp01(df, stock):
    print(stock)
    
    print("1-day lag: ")
    printResult(mape(df['high'],df['maxonedaylag']), 'High')
    printResult(mape(df['low'],df['minonedaylag']), 'Low')
    #print (mae(df['high'],df['maxonedaylag']))
    #print (mae(df['low'],df['minonedaylag']))

    print("SMA5: ")
    printResult(mape(df['high'],df['maxSma5']), 'High')
    printResult(mape(df['low'],df['minSma5']), 'Low')
    #print (mae(df['high'],df['maxSma5']))
    #print (mae(df['low'],df['minSma5']))

    print("SMA10: ")
    printResult(mape(df['high'],df['maxSma10']), 'High')
    printResult(mape(df['low'],df['minSma10']), 'Low')
    #print (mae(df['high'],df['maxSma10']))
    #print (mae(df['low'],df['minSma10']))

    print("SMA20: ")
    printResult(mape(df['high'],df['maxSma20']), 'High')
    printResult(mape(df['low'],df['minSma20']), 'Low')
    #print (mae(df['high'],df['maxSma20']))
    #print (mae(df['low'],df['minSma20']))
    
aux = getPeriod(initialDf_PETR4, TRAIN_BEGIN, TEST_END)
exp1 = exp01(aux)

printExp01(exp1, 'PETR4')

aux = getPeriod(initialDf_VALE3, TRAIN_BEGIN, TEST_END)
exp1 = exp01(aux)

printExp01(exp1, 'VALE3')

PETR4
1-day lag: 
High: 1.35 +- 1.25 %
Low: 1.2 +- 1.12 %
SMA5: 
High: 1.32 +- 1.13 %
Low: 1.26 +- 1.17 %
SMA10: 
High: 2.02 +- 1.7 %
Low: 1.96 +- 1.83 %
SMA20: 
High: 3.24 +- 2.28 %
Low: 3.33 +- 2.45 %
VALE3
1-day lag: 
High: 1.31 +- 0.87 %
Low: 1.3 +- 0.94 %
SMA5: 
High: 1.69 +- 1.15 %
Low: 1.63 +- 0.99 %
SMA10: 
High: 2.54 +- 1.6 %
Low: 2.43 +- 1.53 %
SMA20: 
High: 4.14 +- 2.27 %
Low: 4.08 +- 2.3 %
